# 4 - Chatbot GUI in Databricks
* Notebook by Adam Lang
* Notebook was adopted from the Databricks webinar in June 2024 that streamed on the Databricks YouTube channel.
* This is the 4th notebook and the next step after creating the delta tables, vector endpoint, and vector index table, and serving the model on an endpoint in Databricks. 
* This is one way to create a GUI in databricks but not the only way. 
* Date: 4/30/2025

## 1. Install Dependencies
* Update kernel once completed.

In [0]:
%pip install dbtunnel[gradio] aiohttp
dbutils.library.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/20.3 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 20.2/20.3 MB 121.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/731.2 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 19.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## 2. Set environment Variables

In [0]:
import os 

## 1. databricks token -- if using databricks notebook in databricks
os.environ["DATABRICKS_HOST"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
## 1b. If you are not using a noteook and serving model on cloud then consider using code below
# os.environ['DATABRICKS_TOKEN'] = dbutils.secret.get("demo", "<name of your secret token>")

## 2. Set endpoint of your model
os.environ['API_ENDPOINT'] = "<inference endpoint of your model on databricks>"

## 3. Setup GUI

In [0]:
import itertools
import gradio as gr
import requests
import json
from gradio.themes.utils import sizes


## Function to respond to query 
def respond_to_query(message, history):
    
    if len(message.strip()) == 0:
        return "ERROR the question should not be empty!"
    
    print ("#### Message ####")
    print (message)

    ## get tokens
    local_token = os.getenv('DATABRICKS_TOKEN')
    local_endpoint = os.getenv('API_ENDPOINT')

    if local_token is None or local_endpoint is None:
        return "ERROR missing env variables"
    
    # Add API token to headers
    headers = {
        "Content-Type": 'application/json',
        'Authorization': f"Bearer {local_token}"
    }
    payload = {
        "inputs" : [message]
    }
    try:
        response = requests.post(
            local_endpoint,
            json=payload,
            headers=headers,
            timeout=100
        )
        response_json = response.json()
        print("#### RESPONSE JSON")
        print(response_json)

        response_data = response_json["predictions"][0] # extract field result
        print(response_data)

    except Exception as error:
        response_data = f"ERROR status_code: {type(error).__name__}"
        # + str(response.status_code) + " response:" + response.text

    # print(response.json())
    return response_data


## set theme
theme = gr.themes.Soft(
    text_size=sizes.text_sm,
    radius=sizes.radius_sm,
    spacing_size=sizes.spacing_sm
)

demo = gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(show_label=False, container=False, show_copy_button=True, bubble_full_width=True),
    textbox=gr.Textbox(placeholder="Ask me a question about Home Appliances",
                       container=False,
                       scale=10),
    title="Home Appliances Chatbot",
    description="This chatbot is a demo of how to interact with appliance chatbot",
    examples=[["What do I do if my range burnger won't light?"],],
    cache_examples=False,
    theme=theme,
    retry_btn=None,
    undo_btn=None,
    clear_btn="Clear",
)

In [0]:
## run the demo
from dbtunnel import dbtunnel
dbtunnel.gradio(demo).run()


## once URL is printed out then open it and run demo